In [ ]:
import sys
import sympy as sp

print("Python executable:", sys.executable)
print("SymPy:", sp.__version__)
if "anaconda3" not in sys.executable:
    print("WARNING: you asked to use ~/anaconda3/bin/python. If this is not it, switch the Jupyter kernel.")


In [ ]:
def grad_vec(u, xs):
    """Return grad(u) as a matrix with (i,j) = du_i/dx_j."""
    return sp.Matrix([[sp.diff(u[i], xs[j]) for j in range(len(xs))] for i in range(len(u))])


def sym_grad(u, xs):
    G = grad_vec(u, xs)
    return sp.Rational(1, 2) * (G + G.T)


def div_tensor(T, xs):
    """Return divergence of a 2-tensor T: (div T)_i = sum_j dT_{ij}/dx_j."""
    d = len(xs)
    return sp.Matrix([
        sum(sp.diff(T[i, j], xs[j]) for j in range(d))
        for i in range(d)
    ])


def stress_linear_elasticity(u, xs, lam, mu):
    eps = sym_grad(u, xs)
    d = len(xs)
    return 2 * mu * eps + lam * sp.trace(eps) * sp.eye(d)


def L_elasticity(u, xs, lam, mu):
    """Elasticity operator L(u) = -div sigma(u)."""
    sig = stress_linear_elasticity(u, xs, lam, mu)
    return -div_tensor(sig, xs)


def time_derivative_vec(u, t, order=1):
    if order == 1:
        return sp.Matrix([sp.diff(ui, t) for ui in u])
    if order == 2:
        return sp.Matrix([sp.diff(ui, t, 2) for ui in u])
    raise ValueError("order must be 1 or 2")


def forcing_static(u, xs, lam, mu):
    return sp.simplify(L_elasticity(u, xs, lam, mu))


def forcing_quasistatic(u, xs, t, lam, mu, rho, alpha, beta):
    ut = time_derivative_vec(u, t, 1)
    return sp.simplify(alpha * rho * ut + beta * L_elasticity(ut, xs, lam, mu) + L_elasticity(u, xs, lam, mu))


def forcing_dynamic(u, xs, t, lam, mu, rho, alpha, beta):
    ut = time_derivative_vec(u, t, 1)
    utt = time_derivative_vec(u, t, 2)
    return sp.simplify(rho * utt + alpha * rho * ut + beta * L_elasticity(ut, xs, lam, mu) + L_elasticity(u, xs, lam, mu))


def traction_neumann(u, xs, t, lam, mu, beta, n):
    """t = (sigma(u) + beta*sigma(u_t)) n."""
    ut = time_derivative_vec(u, t, 1)
    sig = stress_linear_elasticity(u, xs, lam, mu)
    sig_t = stress_linear_elasticity(ut, xs, lam, mu)
    return sp.simplify((sig + beta * sig_t) * sp.Matrix(n))

def sympy_to_muparser(expr):
    s = sp.sstr(expr)
    s = s.replace("**", "^")
    return s

def vec_to_muparser(v):
    return [sympy_to_muparser(sp.simplify(e)) for e in list(v)]


## Choose dimension and manufactured displacement

Pick a smooth \(u(x,t)\) that is not a rigid motion, and that is easy to evaluate.

Below are default examples in 2D and 3D; set `dim` accordingly.

In [ ]:
# Travelling wave with spatial decay (Kelvin–Voigt-compatible)

lam, mu, eta, rho, x, y, z, t = sp.symbols("lambda mu eta rho x y z t", real=True)
A, omega, kr, ki = sp.symbols("A omega kr ki", real=True)

xs = sp.Matrix([x, y])

u = sp.Matrix([
    A*sp.exp(-ki*x)*sp.sin(kr*x - omega*t),
    0
])


In [ ]:
# --- Parameters to edit programmatically ---
dim = 2

lam_val = 2.0
mu_val = 1.0
rho_val = 1.0
eta_val = 0.1  # Kelvin–Voigt viscosity (neta in code)
A_val = 0.2
omega_val = 4*sp.pi

dt_val = 0.0125
t0_val = 0.0
t1_val = 0.5

# Kelvin–Voigt spatially decaying wave: compute kr, ki from material + omega
# NOTE: the code uses neta = eta in the damping term, so
# k^2 = rho*omega^2 / (E - i*eta*omega), with E = lambda + 2*mu.
E_val = lam_val + 2*mu_val
c_val = sp.sqrt(E_val / rho_val)
a_val = eta_val * omega_val / E_val
r_val = sp.sqrt(1 + a_val**2)
phi_val = sp.atan(a_val)

kr_val = (omega_val/c_val) * sp.cos(phi_val/2) / sp.sqrt(r_val)
ki_val = (omega_val/c_val) * sp.sin(phi_val/2) / sp.sqrt(r_val)

kr_val = sp.N(kr_val)
ki_val = sp.N(ki_val)

# Build u with symbolic constants A, omega, kr, ki
if dim == 2:
    u = sp.Matrix([A*sp.exp(-ki*x)*sp.sin(kr*x - omega*t), 0])
else:
    u = sp.Matrix([A*sp.exp(-ki*x)*sp.sin(kr*x - omega*t), 0, 0])

# For the KV traveling wave with kr,ki chosen above, RHS is zero
force_zero_rhs = False
if force_zero_rhs:
    f = sp.Matrix([0, 0]) if dim == 2 else sp.Matrix([0, 0, 0])
else:
    ut = time_derivative_vec(u, t, 1)
    utt = time_derivative_vec(u, t, 2)
    # Viscous operator matched to the assembled bilinear form: int eta * eps(u_t):eps(v)
    # A convenient strong-form surrogate is -div(eta * eps(u_t)), i.e. L_elasticity(u_t) with lam=0, mu=eta/2.
    f = sp.simplify(
        rho * utt
        + L_elasticity(u, xs, lam, mu)
        + L_elasticity(ut, xs, 0, eta / 2)
    )

u_expr = vec_to_muparser(u)
v_expr = vec_to_muparser(time_derivative_vec(u, t, 1))
f_expr = vec_to_muparser(f)


In [ ]:
from pathlib import Path

# ---- Programmatic parameters (edit these) ----
params = {
    # Boundary ids (lists or comma-separated strings)
    "dirichlet_ids": [],
    "weak_dirichlet_ids": [0, 1, 2, 3],
    "weak_penalty": 100,

    # Material properties
    "rho": float(rho_val),
    "lam": float(lam_val),
    "mu": float(mu_val),
    "eta": float(eta_val),

    # Wave constants
    "A": float(A_val),
    "omega": float(omega_val),
    "kr": float(kr_val),
    "ki": float(ki_val),

    # Time settings
    "t0": float(t0_val),
    "t1": float(t1_val),
    "dt": float(dt_val),
    "newmark_beta": 0.25,
    "newmark_gamma": 0.5,
}

# Extra constants to make available in expressions (optional)
extra_function_constants = {
    "A": params["A"],
    "omega": params["omega"],
    "kr": params["kr"],
    "ki": params["ki"],
}

# ---- Helpers ----
def ids_to_str(ids):
    if isinstance(ids, str):
        return ids
    return ",".join(str(i) for i in ids)

def format_vec(exprs, dim):
    if dim == 2:
        return f"{exprs[0]}; {exprs[1]}"
    return f"{exprs[0]}; {exprs[1]}; {exprs[2]}"

function_constants = {
    "lambda": params["lam"],
    "mu": params["mu"],
    "eta": params["eta"],
    "rho": params["rho"],
}
function_constants.update(extra_function_constants)
function_constants_str = ", ".join(f"{k}={v}" for k, v in function_constants.items())

params["u_expr"] = format_vec(u_expr, dim)
params["v_expr"] = format_vec(v_expr, dim)
params["f_expr"] = format_vec(f_expr, dim)

# ---- Build parameter file ----
prm_text = f"""
subsection Immersed Problem
  set Dirichlet boundary ids      = {ids_to_str(params["dirichlet_ids"])}
  set Weak Dirichlet boundary ids = {ids_to_str(params["weak_dirichlet_ids"])}
  set Weak Dirichlet penalty coefficient = {params["weak_penalty"]}
  subsection Material properties
    subsection default
      set Density         = {params["rho"]}
      set Lame lambda     = {params["lam"]}
      set Lame mu         = {params["mu"]}
      set Viscosity eta   = {params["eta"]}
    end
  end
  subsection Time dependency
    set Initial time  = {params["t0"]}
    set Final time    = {params["t1"]}
    set Time step     = {params["dt"]}
    set Newmark beta  = {params["newmark_beta"]}
    set Newmark gamma = {params["newmark_gamma"]}
  end
end

subsection Functions
  subsection Dirichlet boundary conditions
    set Function constants = {function_constants_str}
    set Function expression = {params["u_expr"]}
  end
  subsection Exact solution
    set Function constants = {function_constants_str}
    set Function expression = {params["u_expr"]}
  end
  subsection Right hand side
    set Function constants = {function_constants_str}
    set Function expression = {params["f_expr"]}
  end
  subsection Initial displacement
    set Function constants = {function_constants_str}
    set Function expression = {params["u_expr"]}
  end
  subsection Initial velocity
    set Function constants = {function_constants_str}
    set Function expression = {params["v_expr"]}
  end
end
"""

# Write to a file (edit path as needed)
# output_path = Path("manufactured_parameters.prm")
# output_path.write_text(prm_text)

print(prm_text)
# print(f"Wrote: {output_path.resolve()}")
